In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import tomllib
import shapely

In [ ]:
import numpy as np

from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.nddata import CCDData

In [ ]:
import read_iso
import arya

In [ ]:
obj = "yasone1"
julen = True

In [ ]:
r_h = 25 / 60

In [ ]:
r_inner = 40  / 60
r_ring = 2*r_inner

# Data loading

In [ ]:
def get_mag_shift(objname):
    shifts = {}
    with open("../photometry/" + objname + "/panstarrs_shift.toml", "rb") as f:
        shifts = tomllib.load(f)
    return shifts

In [ ]:
def read_catalogue(objname):
    cat = Table.read(f"../photometry/{objname}/allcolours.cat")

    shifts = get_mag_shift(objname)
    cat["GMAG"] -= shifts["g"]
    cat["RMAG"] -= shifts["r"]
    cat["IMAG"] -= shifts["i"]
    cat = cat[(cat["RFLAGS"] < 8 ) & (cat["RFLAGS_WEIGHT"] == 0) ]
    return cat

In [ ]:
def read_catalogue_julen(objname):
    cat = Table.read(f"{objname}_julen.cat")

    return cat

In [ ]:
get_mag_shift(obj)

In [ ]:

if julen:
    cat = read_catalogue_julen(obj)
else:
    cat = read_catalogue(obj)
    # cat["likely_star"] = cat["FLUX_RADIUS"] < 4
    # cat["likely_star"] &= cat["ELLIPTICITY"] < 0.3


In [ ]:
#     cat["likely_star"] = cat["FLUX_RADIUS"] < 4
# plt.hist(cat["FLUX_RADIUS"], bins=np.linspace(0, 10, 30))

In [ ]:
cat

In [ ]:
img = CCDData.read(f"../photometry/{obj}/coadd_median_r/coadd.fits", unit="adu")

In [ ]:
# filt = cat["LIKELY_STAR"]
# fig, axs = plt.subplots(1, 2, subplot_kw=dict(projection=img.wcs), figsize=(6, 4))
# plt.sca(axs[1])
# plt.scatter(cat["X_IMAGE"][filt], cat["Y_IMAGE"][filt], s=1, label="star?")
# plt.scatter(cat["X_IMAGE"][~filt], cat["Y_IMAGE"][~filt], s=0.5, label="galaxy?")
# plt.xlabel("ra")
# plt.ylabel("dec")
# axs[0].set_aspect(1)
# arya.Legend(-1)


# plt.sca(axs[0])
# plt.imshow(img, norm="asinh")
# axs[1].set_aspect(1)

# plt.tight_layout()

In [ ]:
if obj == "yasone1":
    ra0, dec0 = 265.52019, 13.17146
elif obj == "yasone2":
    ra0, dec0 = 262.34921, 6.42302
elif obj == "yasone3":
    ra0, dec0 = 292.96258, -26.44994

In [ ]:
coord0 = SkyCoord(ra0, dec0, unit=u.degree)
r_0 = 25 / 0.254 

# Utilities, etc

In [ ]:
def tangent_axis():
    plt.ylabel(r"$\eta/\textrm{arcmin}$")
    plt.xlabel(r"$\xi/\textrm{arcmin}$")
    plt.xlim(3, -3)
    plt.ylim(-3, 3)
    plt.xticks(np.arange(-3, 4))
    plt.yticks(np.arange(-3, 4))
    plt.gca().set_aspect(1)

In [ ]:
def ri_axis():
    plt.xlabel(r"$r-i$ (mag)")
    plt.ylabel(r"$r$ (mag)")

    plt.xlim(-0.5, 1.5)
    plt.ylim(27, 17)
    plt.gca().set_box_aspect(1)


def gr_axis():
    plt.xlabel(r"$g-r$ (mag)")
    plt.ylabel(r"$g$ (mag)")
    plt.xlim(-0.5, 1.5)
    plt.ylim(27, 17)
    plt.gca().set_box_aspect(1)


In [ ]:
def to_coords(cat):
    return SkyCoord(cat["ALPHA_J2000"], cat["DELTA_J2000"], unit=u.degree)


In [ ]:
def select_centre(cat, coord0, radius):
    filt = coord0.separation(to_coords(cat)) < radius
    return cat[filt]

In [ ]:
def get_upper_radius(radius, radius_bkg):
    return np.sqrt(radius**2 + radius_bkg**2)


In [ ]:
def select_annuli(cat, coord0, radius, radius_bkg):
    radius_upper = get_upper_radius(radius, radius_bkg)
    # print(radius**2, radius_upper**2 - radius_bkg**2)
    seps = coord0.separation(to_coords(cat))

    filt = seps >= radius_bkg
    filt &= seps < radius_upper
    return cat[filt]

In [ ]:
coord0.ra.to("rad").value

In [ ]:
def plot_annuli(radius, radius_bkg):
    circ = plt.Circle((0, 0), radius, color="black", lw=1, fill=False)
    circ2 = plt.Circle((0, 0), radius_bkg, color="black", lw=1, fill=False)
    circ3 = plt.Circle((0, 0), get_upper_radius(radius, radius_bkg), color="black", lw=1, fill=False)
    ax = plt.gca()
        
    ax.add_patch(circ)
    ax.add_patch(circ2)
    ax.add_patch(circ3)

In [ ]:
def to_tangent(coord, coord0):
    alpha = coord.ra.to("rad").value
    delta = coord.dec.to("rad").value
    alpha0 = coord0.ra.to("rad").value
    delta0 = coord0.dec.to("rad").value


    denom = np.sin(delta) * np.sin(delta0) + np.cos(delta) * np.cos(delta0) * np.cos(alpha - alpha0)

    xi_num = np.cos(delta) * np.sin(alpha - alpha0)
    eta_num = np.sin(delta) * np.cos(delta0) - np.cos(delta) * np.sin(delta0) * np.cos(alpha-alpha0) 

    xi = np.rad2deg(xi_num/denom)
    eta = np.rad2deg(eta_num / denom)
    return xi, eta

In [ ]:
xi, eta = to_tangent(to_coords(cat), coord0)
cat["xi"] = xi * 60 * u.arcmin
cat["eta"] = eta * 60 * u.arcmin

In [ ]:
sample_bkg = select_annuli(cat, coord0, r_inner * u.arcmin, r_ring*u.arcmin)

In [ ]:
sample_cen = select_centre(cat, coord0, r_inner * u.arcmin )

In [ ]:
fig, ax = plt.subplots()
plt.scatter(cat["xi"], cat["eta"], s=0.3, color="k")
tangent_axis()
plt.xlim(-5,5)
plt.ylim(-5,5)

## Uncertainty fitting

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def err_model(x, a, b, c):
    return np.log10(np.maximum(1e-12, 10**(x * a + b) + c))

In [ ]:
x = cat["GMAG"]
y = cat["GMAG_ERR"]
filt = np.isfinite(y)
if hasattr(cat["GMAG"], "mask"):
    filt &= ~cat["GMAG"].mask
gpopt, covt = curve_fit(err_model, x[filt], np.log10(y[filt]))
gpopt, np.sqrt(np.diag(covt))

In [ ]:
x = cat["RMAG"]
y = cat["RMAG_ERR"]
filt = np.isfinite(y)
rpopt, covt = curve_fit(err_model, x[filt], np.log10(y[filt]))
rpopt, np.sqrt(np.diag(covt))

In [ ]:
iso_width = 0.05

In [ ]:
def gmag_err(gmag):
    return 10**err_model(gmag, *gpopt)

def rmag_err(mag):
    return 10**err_model(mag, *rpopt)

In [ ]:
fig, axs = plt.subplots(1, 2)

plt.sca(axs[0])
plt.scatter(cat["GMAG"], cat["GMAG_ERR"], s=0.3, lw=0, alpha=0.1, color="k")

x = np.linspace(15, 28)
y = gmag_err(x)
plt.plot(x, y)
plt.yscale("log")
# plt.xlim(0, 1e3)
plt.ylim(1e-3, 2)

plt.ylabel("err")
plt.xlabel("g")

plt.sca(axs[1])
plt.scatter(cat["RMAG"], cat["RMAG_ERR"], s=0.3, lw=0, alpha=0.1, color="k")

x = np.linspace(15, 28)
y = rmag_err(x)
plt.plot(x, y)
plt.yscale("log")
# plt.xlim(0, 1e3)
plt.ylim(1e-3, 2)
plt.xlabel("r")




# CMD by-eye fits

In [ ]:
iso_age = 12e9

if obj == "yasone1":
    dm=16.6
    A_gr=0.2
    A_ri = 0.2
    fe_h="2.00"
elif obj == "yasone2":
    dm=16.6
    A_ri = 0.1
    A_gr = 0.1
    fe_h="2.00"
elif obj == "yasone3":
    dm=15.88
    A_gr=0.1
    A_ri=0.17
    fe_h="1.50"

In [ ]:
isoname = f"../MIST/MIST_v1.2_vvcrit0.0_SDSSugriz/MIST_v1.2_feh_m{fe_h}_afe_p0.0_vvcrit0.0_SDSSugriz.iso.cmd"
# isoname_rot = "../MIST/MIST_v1.2_vvcrit0.4_SDSSugriz/MIST_v1.2_feh_m2.00_afe_p0.0_vvcrit0.4_SDSSugriz.iso.cmd"
# rotation does not matter for ms/rgb

In [ ]:
iso = read_iso.ISOCMD(isoname)[np.log10(iso_age)]

In [ ]:
def plot_iso(dm=10, A=0.1, phase_max=3):
    filt = iso["phase"] < phase_max
    plt.plot(iso["SDSS_g"][filt] - iso["SDSS_r"][filt] + A, iso["SDSS_g"][filt] + dm)


    x_poly, y_poly = make_polygon(iso, gmag_err, dm, A=A, yfilt="SDSS_g", xfilt="SDSS_r")
    plt.plot(x_poly, y_poly, color="k")


In [ ]:
def plot_iso_ri(dm=10, A=0.1, phase_max=3):
    filt = iso["phase"] < phase_max
    plt.plot(iso["SDSS_r"][filt] - iso["SDSS_i"][filt] + A, iso["SDSS_r"][filt] + dm)
    
    x_poly, y_poly = make_polygon(iso, rmag_err, dm, A=A)
    plt.plot(x_poly, y_poly, color="k")


In [ ]:
def plot_iso_bkg_gr(dm=dm, A=A_gr):
    fig, axs = plt.subplots(1, 2)
    x = "GMAG"
    y = "RMAG"

    plt.sca(axs[0])

    plt.scatter(cat[x] - cat[y], cat[x], s=0.3, lw=0, color="grey")
    plt.errorbar(sample_cen[x] - sample_cen[y], sample_cen[x], fmt="o", label="centre")
    plot_iso(dm=dm, A=A)
    
    plt.xlabel("g - r")
    plt.ylabel("g")
    plt.xlim(-1, 1.5)
    plt.ylim(27, 16)

    plt.sca(axs[1])
    plt.scatter(cat[x] - cat[y], cat[x], s=0.3, lw=0, color="grey")
    plt.errorbar(sample_bkg[x] - sample_bkg[y], sample_bkg[x], fmt="o", label="centre")
    plot_iso(dm=dm, A=A)

    plt.xlabel("g - r")
    plt.ylabel("g")

    plt.xlim(-1, 1.5)
    plt.ylim(27, 16)

    fig

In [ ]:
def make_polygon(iso, mag_err, dm, A=A_ri, iso_width=0.05, yfilt="SDSS_r", xfilt="SDSS_i"):
    filt = iso["phase"] < 3

    x = iso[yfilt][filt].data - iso[xfilt][filt].data + A
    y = iso[yfilt][filt].data + dm

    x_min = x - gmag_err(y) - iso_width
    x_max = x + gmag_err(y) + iso_width

    return np.hstack([x_min, x_max[::-1], x_min[0]]), np.hstack([y, y[::-1], y[0]])


In [ ]:
def plot_iso_bkg_ri(dm=dm, A=A_ri):
    fig, axs = plt.subplots(1, 2)
    x = "RMAG"
    y = "IMAG"

    plt.sca(axs[0])

    plt.scatter(cat[x] - cat[y], cat[x], s=0.3, lw=0, color="grey")
    plt.errorbar(sample_cen[x] - sample_cen[y], sample_cen[x], fmt="o", label="centre")
    plot_iso_ri(dm=dm, A=A)

    plt.xlabel("r - i")
    plt.ylabel("i")
    plt.xlim(-1, 1.5)
    plt.ylim(27, 16)


    plt.sca(axs[1])
    plt.scatter(cat[x] - cat[y], cat[x], s=0.3, lw=0, color="grey")
    plt.errorbar(sample_bkg[x] - sample_bkg[y], sample_bkg[x], fmt="o", label="centre")
    plot_iso_ri(dm=dm, A=A)


    plt.xlabel("r - i")
    plt.ylabel("i")

    plt.xlim(-1, 1.5)
    plt.ylim(27, 16)

    fig

In [ ]:

plot_iso_bkg_gr()

In [ ]:

plot_iso_bkg_ri()

In [ ]:
def is_in_poly(x, y, x_poly, y_poly):
    N = len(x)
    poly = shapely.Polygon(np.vstack([x_poly, y_poly]).T)
    
    in_poly = np.full(N, False)
    for i in range(N):
        if np.isfinite(x[i]) and np.isfinite(y[i]):
            point = shapely.Point(x[i], y[i])
            in_poly[i] = point.within(poly)

    return in_poly

In [ ]:
x_poly_ri, y_poly_ri = make_polygon(iso, rmag_err, dm, A=A_ri)

In [ ]:
cmd_filt_ri = is_in_poly(cat["RMAG"].data - cat["IMAG"].data, cat["RMAG"].data, x_poly_ri, y_poly_ri)

In [ ]:
x_poly_gr, y_poly_gr = make_polygon(iso, gmag_err, dm, A=A_gr, xfilt="SDSS_r", yfilt="SDSS_g")

In [ ]:
cmd_filt_gr = is_in_poly(cat["GMAG"].data - cat["RMAG"].data, cat["GMAG"].data, x_poly_gr, y_poly_gr)

In [ ]:

plt.scatter(cat["RMAG"][cmd_filt_ri] - cat["IMAG"][cmd_filt_ri], cat["RMAG"][cmd_filt_ri])
plt.scatter(cat["RMAG"][~cmd_filt_ri] - cat["IMAG"][~cmd_filt_ri], cat["RMAG"][~cmd_filt_ri], s=1)
plot_iso_ri(dm=dm, A=A_ri)
plt.ylim(27, 16)
plt.xlim(-1, 2)

In [ ]:

plt.scatter(cat["GMAG"][cmd_filt_gr] - cat["RMAG"][cmd_filt_gr], cat["GMAG"][cmd_filt_gr])
plt.scatter(cat["GMAG"][~cmd_filt_gr] - cat["RMAG"][~cmd_filt_gr], cat["GMAG"][~cmd_filt_gr], s=1)
plt.scatter(cat["GMAG"][cmd_filt_ri] - cat["RMAG"][cmd_filt_ri], cat["GMAG"][cmd_filt_ri])

plot_iso(dm=dm, A=A_ri)
plt.ylim(27, 16)
plt.xlim(-1, 2)

In [ ]:
plt.scatter(cat["xi"][cmd_filt_gr], cat["eta"][cmd_filt_gr])

In [ ]:
def to_pandas(tbl):
    names = [name for name in tbl.colnames if len(tbl[name].shape) <= 1]
    return tbl[names].to_pandas()

In [ ]:
sns.kdeplot(to_pandas(cat[cmd_filt_gr]), x="xi", y="eta", bw_adjust=0.2)
plt.xlim(-3, 3)
plt.ylim(-3, 3)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(4, 4))
cat_cmd = cat[cmd_filt_gr]

plt.sca(ax[0][0])
plt.scatter(cat["xi"], cat["eta"], s=1, color="grey")
plot_annuli(r_inner, r_ring)
plt.scatter(sample_cen["xi"], sample_cen["eta"], s=1)
plt.scatter(sample_bkg["xi"], sample_bkg["eta"], s=1)
tangent_axis()

plt.sca(ax[0][1])
sns.kdeplot(to_pandas(cat_cmd), x="xi", y="eta",)
plt.scatter(cat_cmd["xi"], cat_cmd["eta"], s=1, color="k")
tangent_axis()


plt.sca(ax[1][0])
plt.scatter(cat["GMAG"] - cat["RMAG"], cat["GMAG"], s=0.3, lw=0, color="grey")
plt.errorbar(sample_cen["GMAG"] - sample_cen["RMAG"], sample_cen["GMAG"], fmt="o", label="centre")
plot_iso(dm=dm, A=A_gr)
gr_axis()
plt.xlim(-1, 2)



plt.sca(ax[1][1])
plt.scatter(cat["GMAG"] - cat["RMAG"], cat["GMAG"], s=0.3, lw=0, color="grey")
plt.errorbar(sample_bkg["GMAG"] - sample_bkg["RMAG"], sample_bkg["GMAG"], fmt="o", label="background annulus")
plot_iso(dm=dm, A=A_gr)
gr_axis()

plt.xlim(-1, 2)

plt.tight_layout()



In [ ]:
sns.kdeplot(to_pandas(cat_cmd), x="xi", y="eta", bw_adjust=0.5)
plt.scatter(cat_cmd["xi"], cat_cmd["eta"], s=1, color=arya.COLORS[2])
plt.gca().set_aspect(1)
plt.xlim(3, -3)
plt.ylim(-3, 3)

In [ ]:
fig = plt.figure(figsize=(5, 5))
Nr, Nc = 2,2

cat_cmd = cat[cmd_filt_ri]

fig.add_subplot(Nr, Nc, 1)
plot_annuli(r_inner, r_ring)
filt = ~cat["IMAG"].mask

plt.scatter(cat["xi"][filt], cat["eta"][filt], s=1, color='grey')
plt.scatter(sample_cen["xi"], sample_cen["eta"], s=1, color=arya.COLORS[2])
plt.scatter(sample_bkg["xi"], sample_bkg["eta"], s=1, color=arya.COLORS[1])
tangent_axis()


fig.add_subplot(Nr, Nc, 2)
sns.kdeplot(to_pandas(cat_cmd), x="xi", y="eta", bw_adjust=0.5)
plt.scatter(cat_cmd["xi"], cat_cmd["eta"], s=1, color="k")
tangent_axis()


fig.add_subplot(Nr, Nc, 3)
plt.scatter(cat["RMAG"] - cat["IMAG"], cat["RMAG"], s=0.3, lw=0, color="grey")
plt.errorbar(sample_cen["RMAG"] - sample_cen["IMAG"], sample_cen["RMAG"], fmt="o", label="centre", color=arya.COLORS[2])

plot_iso_ri(dm=dm, A=A_ri)
ri_axis()
# plt.xlim(-1.2, 1.8)


fig.add_subplot(Nr, Nc, 4)
plt.scatter(cat["RMAG"] - cat["IMAG"], cat["RMAG"], s=0.3, lw=0, color="grey")

plt.errorbar(sample_bkg["RMAG"] - sample_bkg["IMAG"], sample_bkg["RMAG"], fmt="o", label="background annulus", color=arya.COLORS[1])

plot_iso_ri(dm=dm, A=A_ri)
ri_axis()
# plt.xlim(-1.2, 1.8)

fig.suptitle(obj)

plt.tight_layout()

### 

In [ ]:
fig = plt.figure(figsize=(5, 5))
Nr, Nc = 2,2

cat_cmd = cat[cmd_filt_ri]

fig.add_subplot(Nr, Nc, 1)
plot_annuli(r_inner, r_ring)
filt = ~cat["IMAG"].mask

plt.scatter(cat["xi"][filt], cat["eta"][filt], s=1, color='grey')
plt.scatter(sample_cen["xi"], sample_cen["eta"], s=1, color=arya.COLORS[2])
plt.scatter(sample_bkg["xi"], sample_bkg["eta"], s=1, color=arya.COLORS[1])
tangent_axis()


fig.add_subplot(Nr, Nc, 2)
sns.kdeplot(to_pandas(cat_cmd), x="xi", y="eta", bw_adjust=0.5)
plt.scatter(cat_cmd["xi"], cat_cmd["eta"], s=1, color="k")
tangent_axis()


fig.add_subplot(Nr, Nc, 3)
plt.scatter(cat["RMAG"] - cat["IMAG"], cat["RMAG"], s=0.3, lw=0, color="grey")
plt.errorbar(sample_cen["RMAG"] - sample_cen["IMAG"], sample_cen["RMAG"], fmt="o", label="centre", color=arya.COLORS[2])

plot_iso_ri(dm=dm, A=A_ri)
ri_axis()
plt.xlim(-1.2, 1.8)


fig.add_subplot(Nr, Nc, 4)
plt.scatter(cat["RMAG"] - cat["IMAG"], cat["RMAG"], s=0.3, lw=0, color="grey")

plt.errorbar(sample_bkg["RMAG"] - sample_bkg["IMAG"], sample_bkg["RMAG"], fmt="o", label="background annulus", color=arya.COLORS[1])

plot_iso_ri(dm=dm, A=A_ri)
ri_axis()
plt.xlim(-1.2, 1.8)

fig.suptitle(obj)

plt.tight_layout()